In [1]:
import torch

In [2]:
torch.__version__

'2.0.1'

In [3]:
from modules.models.models import SimpleCNN,EPIMind, init_weights
import torchsummary

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [4]:
model = SimpleCNN(encode_method="dna2vec",concat_reverse=False,
                  init_method="xavier_normal")

In [4]:
model = EPIMind(concat_reverse=False,embedding_dim=100,
                 num_heads=8, num_layers=8, num_hiddens=128, ffn_num_hiddens=256,verbose=False)

In [44]:
model = torch.nn.Transformer()

In [36]:
a = model(torch.randn((1, 32, 512)),torch.randn((1, 32, 512)))
print(a.shape)

torch.Size([1, 32, 512])


In [45]:
torchsummary.summary(model, [(32, 512),(32, 512)],batch_size=16,device="cpu")
print('parameters_count:',count_parameters(model))

AttributeError: 'NoneType' object has no attribute 'size'

In [20]:
# from thop import profile
model = SimpleCNN(encode_method="dna2vec",concat_reverse=False,
                  init_method="xavier_normal")

In [6]:
from ptflops import get_model_complexity_info
import re

with torch.cuda.device(0):
  # net = models.densenet161()
  macs, params = get_model_complexity_info(model, (6000, 200), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
flops = eval(re.findall(r'([\d.]+)', macs)[0])*2 
# Extract the unit 
flops_unit = re.findall(r'([A-Za-z]+)', macs)[0][0]

EPIMind(
  4.25 M, 100.000% Params, 3.05 GMac, 100.000% MACs, 
  (enhancer_conv): Sequential(
    460.93 k, 10.844% Params, 1.83 GMac, 60.007% MACs, 
    (0): Conv1d(460.93 k, 10.844% Params, 1.83 GMac, 59.990% MACs, 100, 128, kernel_size=(36,), stride=(1,), padding=valid)
    (1): ReLU(0, 0.000% Params, 507.52 KMac, 0.017% MACs, )
  )
  (promoter_conv): Sequential(
    460.93 k, 10.844% Params, 905.98 MMac, 29.739% MACs, 
    (0): Conv1d(460.93 k, 10.844% Params, 905.72 MMac, 29.730% MACs, 100, 128, kernel_size=(36,), stride=(1,), padding=valid)
    (1): ReLU(0, 0.000% Params, 251.52 KMac, 0.008% MACs, )
  )
  (enhancer_maxpool): Sequential(
    0, 0.000% Params, 507.52 KMac, 0.017% MACs, 
    (0): MaxPool1d(0, 0.000% Params, 507.52 KMac, 0.017% MACs, kernel_size=20, stride=20, padding=0, dilation=1, ceil_mode=False)
  )
  (promoter_maxpool): Sequential(
    0, 0.000% Params, 251.52 KMac, 0.008% MACs, 
    (0): MaxPool1d(0, 0.000% Params, 251.52 KMac, 0.008% MACs, kernel_size=20, stri

In [7]:
print('Computational complexity: {:<8}'.format(macs)) 
print('Computational complexity: {} {}Flops'.format(flops, flops_unit)) 
print('Number of parameters: {:<8}'.format(params))

Computational complexity: 3.05 GMac
Computational complexity: 6.1 GFlops
Number of parameters: 4.25 M  


In [3]:
import torch 
import torch.nn as nn
import numpy as np
 
#第一种方式
class GlobalMaxPool1d(nn.Module):
    def __init__(self):
        super(GlobalMaxPool1d,self).__init__()
    def forward(self, x):
        return torch.max_pool1d(x,kernel_size=x.shape[2])
 
a = torch.tensor(np.arange(24),dtype=torch.float).view(2,3,4).cuda()
print(a.shape)
gmp1 = GlobalMaxPool1d()
print(gmp1(a).shape)
 
 
#第二种方式
gmp2 = torch.nn.AdaptiveMaxPool1d(1)
print(gmp2(a).shape)

torch.Size([2, 3, 4])
torch.Size([2, 3, 1])
torch.Size([2, 3, 1])


In [7]:
flatten = nn.Flatten()
print(flatten(a).shape)

torch.Size([2, 12])
